# Import modelflow

In [23]:
from modelclass import model
model.widescreen()

# Load model and data, and run 

In [27]:
if not 'masia' in locals():
    print('Load model')
    masia,baseline = model.modelload('Asia.pcim',run=1,silent=1)

In [28]:
from inject import inject
inject(masia)

# Variables to inputwidget

In [11]:
outvar = 'GDI GCARBR CO2 LNN'.split()
displayvar = 'WLD_CO2 ASP_CO2 G20_CO2 '
slidedef_all =  {'ALL' : {'var': ' '.join([f'{country}_GCARBR_A' for country in  masia.countries_GCARBR_A]), 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10}}
slidedef_countries = {masia.iso_dict.get(country,country) : {'var': f'{country}_GCARBR_A', 'value': 0.0,'min':0,'max':100,'op':'+impulse','step':10} for country in  masia.countries_GCARBR_A}

slidedef = {**slidedef_all,**slidedef_countries}

# Define  experiment

In [12]:
input = masia.inputwidget(2021,2050,slidedef=slidedef
                   ,showout=True,varpat=displayvar,trans=masia.var_description,showvar=True)

# Analyze results

In [19]:
masia['AFG_EXP'].frml

AFG_EXP : FRML <Z,EXO> AFG_EXP =( AFG_EXPE+AFG_EXPH+AFG_EXPSP+AFG_OGC+AFG_OGI+AFG_GIP + AFG_EXP_A )*(1-AFG_EXP_D)+AFG_EXP_X*AFG_EXP_D$


In [22]:
input

In [21]:
masia.AFG_EXPe

Endogeneous: AFG_EXPE: General government expenditure on environmental protection, Billions National Currency ,Afghanistan 
Formular: FRML <Z,EXO> AFG_EXPE =( AFG_EXPE(-1)*(AFG_YFT/AFG_YFT(-1))*(AFG_YED/AFG_YED(-1)) + AFG_EXPE_A )*(1-AFG_EXPE_D)+AFG_EXPE_X*AFG_EXPE_D$

AFG_EXPE  : General government expenditure on environmental protection, Billions National Currency ,Afghanistan
AFG_EXPE_A: Add factor:General government expenditure on environmental protection, Billions National Currency ,Afghanistan
AFG_EXPE_D: Exo dummy:General government expenditure on environmental protection, Billions National Currency ,Afghanistan
AFG_EXPE_X: Exo value:General government expenditure on environmental protection, Billions National Currency ,Afghanistan
AFG_YED   : Deflator for  GDP, National Currency, 2015 =100 ,Afghanistan
AFG_YFT   : Trend output, Constant 2015 prices, Billions National Currency ,Afghanistan

Values : 
              2021          2022          2023          2024          2025  \
B

In [15]:
masia.GBR_YEN_DOLLAR

Endogeneous: GBR_YEN_DOLLAR:  
Formular: FRML <Z,EXO> GBR_YEN_DOLLAR =( GBR_YER_DOLLAR*GBR_YED/GBR_EXR*(GBR_EXR_VALUE_2015) + GBR_YEN_DOLLAR_A )*(1-GBR_YEN_DOLLAR_D)+GBR_YEN_DOLLAR_X*GBR_YEN_DOLLAR_D$

GBR_YEN_DOLLAR  : 
GBR_EXR         : Exchange rate (national currency / US$) ,UK
GBR_YED         : Deflator for  GDP, National Currency, 2015 =100 ,UK
GBR_YEN_DOLLAR_A: Add factor:GBR_YEN_DOLLAR
GBR_YEN_DOLLAR_D: Exo dummy:GBR_YEN_DOLLAR
GBR_YEN_DOLLAR_X: Exo value:GBR_YEN_DOLLAR

Values : 
              2021          2022          2023          2024          2025  \
Base  3.452524e+03  3.573618e+03  3.703991e+03  3.841393e+03  3.996431e+03   
Last  3.452524e+03  3.573618e+03  3.703991e+03  3.841393e+03  3.996431e+03   
Diff  5.840320e-09  8.860297e-09  5.387847e-09  4.836238e-09 -1.309945e-08   

              2026          2027          2028          2029          2030  \
Base  4.156558e+03  4.324556e+03  4.500601e+03  4.684732e+03  4.877024e+03   
Last  4.156558e+03  4.324556e+03  4.5